In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

Load anndata

In [2]:
adata = sc.read_h5ad("test.h5ad")
# lognorm should be the default (expected)
adata.X = adata.layers['logcounts']

Parameters

In [3]:
groupby='label'

In [4]:
de_method='wilcoxon'

In [5]:
resource_name = 'consensus'

In [6]:
complex_policy = 'min'

In [7]:
key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [8]:
verbose = False

In [9]:
# only relevant if perms are done
n_perms = 1000
seed = 69

Run LIANA PIPE

In [10]:
from liana import liana_pipe

In [11]:
lr_res = liana_pipe(adata=adata, 
                    groupby=groupby, 
                    resource_name=resource_name,
                    de_method=de_method,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    resource=None)

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/dbdimitrov/Repos/liana-py/liana/utils/reassemble_complexes.py:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  lr_res = lr_res.obj.merge(cols_dict[col], left_on=left_on,


In [12]:
lr_res

,ligand,ligand_complex,ligand_means,mat_mean,receptor,receptor_complex,receptor_means,source,target,ligand_means_sums,receptor_means_sums
0,LGALS9,LGALS9,0.374320,0.199056,PTPRC,PTPRC,0.761533,B,B,0.534913,3.409275
1,CD22,CD22,0.554265,0.199056,PTPRC,PTPRC,0.761533,B,B,0.672466,3.409275
2,LGALS1,LGALS1,0.611330,0.199056,PTPRC,PTPRC,0.761533,B,B,3.588291,3.409275
3,LGALS9,LGALS9,0.374320,0.199056,CD44,CD44,0.655391,B,B,0.534913,2.103060
4,ADAM10,ADAM10,0.119797,0.199056,CD44,CD44,0.655391,B,B,0.362591,2.103060
...,...,...,...,...,...,...,...,...,...,...,...
4282,CALM1,CALM1,2.516959,0.199056,KCNN4,KCNN4,0.000000,NK,NK,6.145011,0.121264
4283,PTPN6,PTPN6,0.652221,0.199056,CD300LF,CD300LF,0.000000,NK,NK,2.501501,0.052014
4284,NUCB2,NUCB2,0.156951,0.199056,ERAP1,ERAP1,0.162055,NK,NK,0.386504,0.209576
4285,SOCS2,SOCS2,0.230338,0.199056,EPOR,EPOR,0.000000,NK,NK,0.372007,0.050942


CellPhoneDB

In [ ]:
from liana import cellphonedb

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

In [ ]:
adata.uns['liana_res']

NATMI

In [ ]:
from liana import natmi

In [ ]:
adata = natmi(adata, groupby='label')

In [ ]:
adata.uns['liana_res']

SCA

In [16]:
from liana import singlecellsignalr as sca

In [17]:
adata = sca(adata, groupby='label')

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/dbdimitrov/Repos/liana-py/liana/utils/reassemble_complexes.py:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  lr_res = lr_res.obj.merge(cols_dict[col], left_on=left_on,


In [19]:
adata.uns['liana_res'].sort_values('lr_means', ascending=True)

,ligand,ligand_complex,ligand_means,mat_mean,receptor,receptor_complex,receptor_means,source,target,lr_means
2485,B2M,B2M,5.810702,0.199056,CD3D,CD3D,2.190063,NK,CD8 T,0.947149
2008,B2M,B2M,5.599935,0.199056,CD3D,CD3D,2.190063,CD8 T,CD8 T,0.946217
4110,B2M,B2M,5.810702,0.199056,CD247,CD247,1.937585,NK,NK,0.943998
1531,B2M,B2M,5.080301,0.199056,CD3D,CD3D,2.190063,B,CD8 T,0.943684
3635,B2M,B2M,5.599935,0.199056,CD247,CD247,1.937585,CD8 T,NK,0.943013
...,...,...,...,...,...,...,...,...,...,...
1958,IL18BP,IL18BP,0.000000,0.199056,IL6R,IL6R,0.000000,CD8 T,CD8 T,0.000000
1963,TNFSF12,TNFSF12,0.259703,0.199056,TNFRSF12A,TNFRSF12A,0.000000,CD8 T,CD8 T,0.000000
1967,BTLA,BTLA,0.000000,0.199056,TNFRSF14,TNFRSF14,0.294117,CD8 T,CD8 T,0.000000
1970,ICOSLG,ICOSLG,0.000000,0.199056,CD28,CD28,0.206916,CD8 T,CD8 T,0.000000


In [ ]:
lr_res[(lr_res.ligand_complex=='TGFB1') & (lr_res.receptor_complex=='ACVR1_TGFBR1_TGFBR2')].sort_values('lrscore', ascending=False)

In [ ]:
lr_res

Connectome

try w/ real data

In [ ]:
adata = sc.datasets.pbmc3k_processed()
labels = adata.obs.louvain
cells = adata.obs_names

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata = adata[[x in cells for x in adata.obs_names]]

In [ ]:
adata.obs['label'] = labels

In [ ]:
sc.pp.log1p(adata)

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

Calculate means_sums for NATMI

In [ ]:
def _sum_means(lr_res, what, on):
    return lr_res.join(lr_res.groupby(on)[what].sum(), on=on, rsuffix='_sums')

In [ ]:
lr_res = _sum_means(lr_res, what='ligand_means', on=['ligand_complex', 'receptor_complex', 'target'])
lr_res = _sum_means(lr_res, what='receptor_means', on=['ligand_complex', 'receptor_complex', 'source'])

In [ ]:
lr_res

NATMI fun

Both magnitude and specificity! Returned to unpack!!!

In [ ]:
lr_res[['expr_prod','spec_weight']] = lr_res.apply(_natmi_score, axis=1, result_type="expand")

In [ ]:
lr_res[lr_res['edge_specificity']==1].sort_values('ligand')

#### LIANA PIPE LINE BY LINE

In [ ]:
from liana.utils.pre import check_mat, check_if_covered, format_vars, filter_resource
from liana.resource import select_resource
from liana.resource.select_resource import explode_complexes
from liana.steady.liana_pipe import _get_lr
from liana.utils.reassemble_complexes import reassemble_complexes
from scipy.sparse import csr_matrix
from liana.scores import get_means_perms

I need to double check if every step makes sense, as I build unit tests!!!

For example, are there any duplicated rows (source, target, ligand_complex, receptor_complex)

Double check again if correct subunit is kept, etc

In [ ]:
adata.X = check_mat(adata.X, True)

In [ ]:
# Define idents col name
adata.obs.label = adata.obs[groupby]

Load resource

In [ ]:
resource = select_resource(resource_name='consensus')

In [ ]:
# Decomplexify
resource = explode_complexes(resource)

In [ ]:
# Filter Resource
resource = filter_resource(resource, adata.var_names)

Create entities

In [ ]:
entities = np.union1d(np.unique(resource["ligand"]), np.unique(resource["receptor"]))

Check overlap between resource and adata

In [ ]:
check_if_covered(entities, adata.var_keys)

Get global mean for SCA before filtering

In [ ]:
if 'mat_mean' in add_cols: # SHOULD BE METHOD NAME!
    adata.uns['mat_mean'] = np.mean(adata.X)

In [ ]:
# Filter to only include the relevant genes
adata = adata[:,np.intersect1d(entities, adata.var.index)]
adata

Get Stats

In [ ]:
lr_res = _get_lr(adata, resource, key_cols + complex_cols + add_cols, de_method)

In [ ]:
lr_res

Recomplexify

In [ ]:
# If I want to implement anything else but the min as complex policy, I would need to change this:
# temp = temp[[x=="INHBA_INHBB" for x in temp['ligand_complex']]].sort_values('receptor_complex')
# temp = temp[[x=="ACVR1B_ACVR2A" for x in temp['receptor_complex']]]

In [ ]:
lr_res = reassemble_complexes(lr_res, key_cols, complex_cols, complex_policy)

### SCORES

In [ ]:
from liana.steady.Method import Method, MethodMeta
from liana.steady.scores.cellphonedb import _cpdb_score

CELLPHONEDB

In [ ]:
_cellphonedb = MethodMeta(method_name = "cellphonedb",
                          complex_cols = ['ligand_means', 'receptor_means'], 
                          add_cols = ['ligand', 'receptor'],
                          fun = _cpdb_score,
                          magnitude = 'lr_means',
                          specificity = 'pvals',
                          permute = True,
                          reference = 'Efremova et al., 2020')

In [ ]:
cellphonedb = Method(_SCORE=_cellphonedb)

In [ ]:
adata = cellphonedb(adata, groupby, seed=666)

In [ ]:
adata.uns['liana_res']

# ^ related to me subsetting the matrix, rather than the anndata object - just replace x /w andata

In [ ]:
adata.X =check_mat(adata.X)

In [ ]:
_sum = np.sum(adata.X.data[0:100])

In [ ]:
np.isfinite(adata.X.data)

In [ ]:
np.sum(x != 0, axis=0).A1 == 0

In [ ]:
# ScoreClass(method_name='cellphonedb',
#           method_cols = ['ligand_means', 'receptor_means'],
#           score_fun = )

In [ ]:
if cpdb_score.permute:
    perms, ligand_pos, receptor_pos, labels_pos = get_means_perms(adata=adata, lr_res=lr_res, n_perms=1000, seed=69)

In [ ]:
%%time
lr_res[[cpdb_score.magnitude, cpdb_score.specificity]] = lr_res.apply(cpdb_score.fun, axis=1, result_type="expand", perms=perms, ligand_pos=ligand_pos, receptor_pos=receptor_pos, labels_pos=labels_pos)

In [ ]:
lr_res

In [ ]:
lr_res

logFC re-implement

Magnitude as None

In [ ]:
lr_res['logfc'] = lr_res[['ligand_logfoldchanges', 'receptor_logfoldchanges']].mean(1)

In [ ]:
lr_res.sort_values(by='logfc', key=abs, ascending=False)

Re-implement Connectome

Both Specificity and Magnitude

In [ ]:
lr_res['edge_weight'] = lr_res[['ligand_zscores', 'receptor_zscores']].mean(1)